Internship schedule: 4 weeks, 3 days per week.

Each morning, there will be a 20-minute session to explain the tasks for the day.
In the last 30 minutes before the end of the day, we will review the day’s practice and answer any questions.

# Week 1

## Day 1

### Dev Environment Preparation

### Concept of Version Control

#### git learning

1.  Register Github 
2.  Create Project
3.  git environment 

    windows：https://git-scm.com/install/windows
    
    macOS：https://git-scm.com/install/mac （can use homebrew to install）

4.  Create a new Python notebook file, for example practise.ipynb, and use it to practise [Markdown syntax](https://www.markdownguide.org/basic-syntax/).

5.  Practice git command, including git add, git commit, git push, git pull 

#### markdown syntax

#### jupyter

    create a new repository Internship-Project-Analysis

## Day 2

### Go

#### Document: https://go.dev/

install: https://go.dev/dl/

a tour of Go: https://go.dev/tour/welcome/1

### Vue

#### Document: https://vuejs.org/

#### UI Framework: https://vuetifyjs.com/en/

Day 2 - Day 3 Main Practice Focus:

Set up the Golang and Vue development environments

Practice with "A Tour of Go" to get familiar with Golang syntax

## Day 3

Task 1: Create a new front-end project and upload it to GitHub

Task 2: Save Data to SQLite [Graduate Employment Survey Data](https://data.gov.sg/datasets?topics=education&resultId=d_3c55210de27fcccda2ed0c63fdd2b352) 

### init front-end project

#### node

install-https://nodejs.org/en/download

#### create vue.js project-[Quick start](https://vuejs.org/guide/quick-start.html)

```
$ npm create vue@latest
```

Then push to github repository



# Week 2

## Day 1


Init Project
```
npm create vue@latest

cd Internship-Project-Web
npm install
npm run format
npm run dev
```

Or just fork this repo

```
git clone git@github.com:zhaoyingx/Internship-Project-Web.git
```

Task 1 - Update cards contents according to our product

Task 2 - add 4 charts and 1 table in DsshboardView.vue

### Documentation

Vuetify

Components: https://vuetifyjs.com/en/components/all/#containment

EChart

Demo: https://echarts.apache.org/examples/en/index.html

API: https://echarts.apache.org/en/api.html#echarts

## Day 2

Practice Vuetify UI Components, design and build website

## Day 3

Server develop

Reference: 
```
git clone git@github.com:zhaoyingx/Internship-Project-Server.git

go run main.go
```

# Week 3

> **Note:** Intern on leave this week.

# Week 4

## Day 1

### API Docker image

1.  Learn about Docker
2.  Write Dockerfile
3.  Build docker image
    ```
    docker build -t internship-api .
    ```
4.  Run
```
    $ docker run -p 8081:8081 internship-api
    [GIN-debug] [WARNING] Creating an Engine instance with the Logger and Recovery middleware already attached.

    [GIN-debug] [WARNING] Running in "debug" mode. Switch to "release" mode in production.
    - using env:	export GIN_MODE=release
    - using code:	gin.SetMode(gin.ReleaseMode)

    [GIN-debug] GET    /health                   --> api-template/routes.SetupRouter.func1 (4 handlers)
    [GIN-debug] GET    /api/v1/graduate-employment --> api-template/controllers.QueryGraduateEmployment (4 handlers)
    2025/12/07 17:08:52 服务器启动于 :8081
    [GIN-debug] [WARNING] You trusted all proxies, this is NOT safe. We recommend you to set a value.
    Please check https://github.com/gin-gonic/gin/blob/master/docs/doc.md#dont-trust-all-proxies for details.
    [GIN-debug] Listening and serving HTTP on :8081
```

### Web Docker image

1.  Write Dockerfile
2.  Build image
```
    docker build -t internship-web .                                      
```
3.  Run
```
    $ docker run -p 8082:80 internship-web

```

## Day 2

#### local deploy

[kind](https://kind.sigs.k8s.io/docs/user/quick-start/#installation)

Step

1.  launch docker desktop
2.  touch kind-config.yaml
    ```Yaml
    kind: Cluster
    apiVersion: kind.x-k8s.io/v1alpha4
    nodes:
    - role: control-plane
        extraPortMappings:
        - containerPort: 30000   # Web NodePort
            hostPort: 30000
            protocol: TCP
        - containerPort: 30010   # API NodePort
            hostPort: 30010
            protocol: TCP

    ```
3.  create cluster with kind-config.yaml
```bash
    kind create cluster --name kind-test-1 --config kind-config.yaml
```
```
    Creating cluster "kind-test-1" ...
    ✓ Ensuring node image (kindest/node:v1.34.0) 🖼
    ✓ Preparing nodes 📦
    ✓ Writing configuration 📜
    ✓ Starting control-plane 🕹️
    ✓ Installing CNI 🔌
    ✓ Installing StorageClass 💾
    Set kubectl context to "kind-kind-test-1"
    You can now use your cluster with:

    kubectl cluster-info --context kind-kind-test-1

    Thanks for using kind! 😊
```

4.  prepare yaml for api service

    api-deployment.yaml
    ```Yaml
    apiVersion: apps/v1
    kind: Deployment
    metadata:
    name: api-deployment
    spec:
    replicas: 1
    selector:
        matchLabels:
        app: internship-api
    template:
        metadata:
        labels:
            app: internship-api
        spec:
        containers:
            - name: internship-api
            image: internship-api:latest
            imagePullPolicy: IfNotPresent
            ports:
                - containerPort: 8081
            env:
                - name: ENV
                value: "production"

    ```

    api-service.yaml
    ```Yaml
    apiVersion: v1
    kind: Service
    metadata:
    name: api-service
    spec:
    type: NodePort
    selector:
        app: internship-api
    ports:
        - port: 8081
        targetPort: 8081
        nodePort: 30010
    ```


5.  Update web request API

    First, check your own IP, on macOS, can use ```ipconfig getifaddr en0```, on windows, can use ```ipconfig```
    
    Second, in your web project, find the .env file, then change the api address like this, ```VITE_API_URL=http://10.196.4.202:30010/api/v1```
    Notice, the ip address should be updated with your own IP

    Third, build web Image again. 

6.  prepare yaml for web service

    web-deployment.yaml
    ```Yaml
    apiVersion: apps/v1
    kind: Deployment
    metadata:
    name: web-deployment
    spec:
    replicas: 1
    selector:
        matchLabels:
        app: internship-web
    template:
        metadata:
        labels:
            app: internship-web
        spec:
        containers:
            - name: internship-web
            image: internship-web:latest
            imagePullPolicy: IfNotPresent
            ports:
                - containerPort: 80
    ```

    web-service.yaml
    ```Yaml
    apiVersion: v1
    kind: Service
    metadata:
    name: web-service
    spec:
    type: NodePort
    selector:
        app: internship-web
    ports:
        - port: 80           
        targetPort: 80     # containerPort
        nodePort: 30000    

    ```



7.  load image into cluster
    for example, the cluster name is kind-test-1
    ```bash
    kind load docker-image internship-api --name kind-test-1
    kind load docker-image internship-web --name kind-test-1
    ```

8.  kubectl apply yaml
    ```bash
    kubectl apply -f api-deployment.yaml
    kubectl apply -f api-service.yaml
    kubectl apply -f web-deployment.yaml
    kubectl apply -f web-service.yaml
    ```

9.  load web page from your phone, for example, load http://10.196.4.202:30000/dashboard
